# noecho

> For running multi-line Stata code without echoing the commands

We can run multi-line Stata code as a [Stata program](https://www.stata.com/manuals/u18.pdf) to prevent the echo of commands. But this approach requires special care for handling both local macros and valid Stata code that cannot be run inside a Stata program. 

(Note: An alternative approach, prepending each Stata command with `noisily` and placing them inside a `quietly` block, was [rejected at a certain point](https://github.com/ticoneva/pystata-kernel/pull/18#issuecomment-1275223836) as more difficult overall, but it may be worth revisiting if unanticipated issues with the current run-as-program approach arise.) 

In [ ]:
#| default_exp noecho
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from nbstata.code_utils import break_out_prog_blocks
from nbstata.stata import run_direct, set_local, run_single
from nbstata import stata_more as sm 
from textwrap import dedent
import re

## Run as program but with main-scope locals

We aim to run Stata code inside a Stata program, yet handling locals as if the code were run normally (rather than inside the program scope).

We accomplish this by first grabbing any previously-defined locals (with `stata_more.get_local_dict`) and artificially defining their values at the start of the program (using `stata_more.locals_code_from_dict`). 

Then we also want to artificially transfer any locals created within the program to the outside scope. We do this by making the program an [s-class program](https://www.stata.com/help.cgi?sreturn) and storing any locals present at the end of the program via `sreturn`. These locals can then be quietly defined in the main scope after the program run has completed.

In [ ]:
#| export
def _run_as_program_w_locals_sreturned(std_code):
    sreturn_code = dedent("""\
        
        mata : st_local("temp_nbstata_all_locals", invtokens(st_dir("local", "macro", "*")'))
        foreach lname in `temp_nbstata_all_locals' {
            sreturn local `lname' "``lname''"
        }
        """)
    store_new_locals_code = ("sreturn clear\n" 
                             + std_code
                             + sreturn_code)                          
    sm.run_as_program(store_new_locals_code, "sclass")

In [ ]:
#| hide
#| eval: false
from nbstata.config import launch_stata

In [ ]:
#| hide
#| eval: false
launch_stata(splash=False)

In [ ]:
#| hide
#| eval: False
_run_as_program_w_locals_sreturned('local test3 "3"')
run_single("sreturn list")


macros:
              s(test3) : "3"


In [ ]:
#| export
#| hide
parse_sreturn = re.compile(
    r'^\s*?(?:\ss\((?P<name>\w+)\) : \"(?P<value>.+)\"\s)', flags=re.MULTILINE
).findall

In [ ]:
#| hide
output = """
macros:
              s(test1) : "blah"
              s(test2) : "blah blah"

"""
parse_sreturn(output)

[('test1', 'blah'), ('test2', 'blah blah')]

In [ ]:
#| export
def _local_dict_from_sreturn(sreturn_output):
    matches = parse_sreturn(sreturn_output)
    return {m[0]: m[1] for m in matches}

In [ ]:
#| hide
_local_dict_from_sreturn(output)

{'test1': 'blah', 'test2': 'blah blah'}

In [ ]:
#| export
def _after_local_dict():
    sreturn_output = sm.diverted_stata_output_quicker("sreturn list")
    return _local_dict_from_sreturn(sreturn_output)

In [ ]:
#| export
def _restore_locals_and_clear_sreturn():
    for lname, value in _after_local_dict().items():
        set_local(lname, value)
    run_single("sreturn clear", show_exc_warning=False)

In [ ]:
#| export
#| hide
pre = (
    r'(cap(t|tu|tur|ture)?'
    r'|qui(e|et|etl|etly)?'
    r'|n(o|oi|ois|oisi|oisil|oisily)?)')
kwargs = {'flags': re.MULTILINE}
local_def_in = re.compile(
    r"(^({0} )*(loc(a|al)?|tempname|tempvar|tempfile|gettoken|token(i|iz|ize)?)\s)|st_local\(".format(pre),
    **kwargs,
).search

In [ ]:
#| hide
from fastcore.test import test_eq

In [ ]:
#| hide
test_eq(bool(local_def_in("sysuse auto")), False)
test_eq(bool(local_def_in("loc auto=1")), True)
test_eq(bool(local_def_in("qui n cap local auto=1")), True)
test_eq(bool(local_def_in("list local auto")), False)
test_eq(bool(local_def_in("tempfile file1")), True)
test_eq(bool(local_def_in("capture token file1")), True)
test_eq(bool(local_def_in("mata: st_local(test1, 2)")), True)

In [ ]:
#| export
def run_as_program_w_locals(std_code, local_dict=None):
    if local_dict is None:
        local_dict = sm.get_local_dict()
    locals_code = sm.locals_code_from_dict(local_dict)
    if not local_def_in(std_code):
        sm.run_as_program(f"""{locals_code}\n{std_code}""")
    else:
        _run_as_program_w_locals_sreturned(f"""{locals_code}\n{std_code}""")
        _restore_locals_and_clear_sreturn()

In [ ]:
#| eval: false
sm.run_sfi(dedent("""
    macro drop _all
    local local1 = 1
    local local2 "two"
    local local3 `""3""' """))
run_as_program_w_locals("""disp `"`local1' `local2' `local3'"' """)

1 two "3"


In [ ]:
#| eval: false
code = '''\
local test1 "blah blah"
local test2 "blah"
'''
run_as_program_w_locals("""disp `"`local1' `local2' `local3'"' \n""" + code)
test_eq(sm.get_local_dict(), 
        {'test2': 'blah',
         'test1': 'blah blah',
         'local1': '1',
         'local2': 'two',
         'local3': '"3"'})

1 two "3"


In [ ]:
#| export
def run_non_prog_noecho(std_non_prog_code, run_as_prog=run_as_program_w_locals):
    if len(std_non_prog_code.splitlines()) <= 1:  # to keep it simple when we can
        run_direct(std_non_prog_code, quietly=False, inline=True, echo=False)
    else:
        run_as_prog(std_non_prog_code)

In [ ]:
#| eval: false
run_non_prog_noecho('disp "test 1"')

test 1


In [ ]:
#| eval: false
two_lines_of_code = dedent('''\
    disp "test 1"
    disp "test 2"
    ''')
run_non_prog_noecho(two_lines_of_code)

test 1
test 2


## Run any Stata code noecho

Certain valid Stata code (referred to here as `prog_code` because it defines Stata programs--or runs mata or python code blocks) cannot be run inside a Stata program, so we need to identify such code and run it separately. 

In [ ]:
#| export
def run_prog_noecho(std_prog_code):
    if std_prog_code.splitlines()[0] in {'mata', 'mata:'}:  # b/c 'quietly' blocks mata output
        run_direct(std_prog_code, quietly=False, inline=True, echo=False)
    else:
        run_direct(std_prog_code, quietly=True, inline=True, echo=False)

Other programs (that is, Stata's `program define`, as well as [mata](https://www.stata.com/manuals/m-1first.pdf) or [python](https://www.stata.com/stata-news/news35-3/python-blogs/) blocks) cannot be defined/run within a Stata program, however. Instead, we will just run them directly, quietly to prevent echo, except for the case of mata programs, in which case `quietly` would block the output.

In [ ]:
#| eval: false
#| hide
sm.run_as_program("capture program drop display1")

In [ ]:
#| eval: false
prog_block_code = dedent("""\
    program define display1
        disp "display1 output"
    end
    """)
run_prog_noecho(prog_block_code)
run_single("display1")

display1 output


In [ ]:
#| eval: false
python_block_code = dedent("""\
    python:
    print("hello")
    end
    """)
run_prog_noecho(python_block_code)

hello


In [ ]:
#| eval: false
mata_block_code = dedent("""\
    mata:
    display("hello")
    end
    """)
run_prog_noecho(mata_block_code)


. mata:
------------------------------------------------- mata (type end to exit) -----
: display("hello")
hello

: end
-------------------------------------------------------------------------------

. 


In [ ]:
#| export
def run_noecho(code, sc_delimiter=False, run_as_prog=run_as_program_w_locals):
    """After `break_out_prog_blocks`, run each prog and non-prog block noecho"""
    for block in break_out_prog_blocks(code, sc_delimiter):
        if block['is_prog']:
            run_prog_noecho(block['std_code'])
        else:
            run_non_prog_noecho(block['std_code'], run_as_prog=run_as_prog)

In [ ]:
#| eval: false
run_noecho(dedent('''\
    capture program drop ender
    program define ender
        disp "ender output"
    end
    capture program drop display2
    program define display2
        ender
    end
    display2
    '''))

ender output


In [ ]:
#| eval: false
run_noecho(dedent("""\
    disp `"`local1' `local2' `local3'"'
    disp `"`local1' `local2' `local3' `test1'"'
    """), run_as_prog=run_as_program_w_locals)

1 two "3"
1 two "3" blah blah


In [ ]:
#| eval: false
code = """\
local local1 "foo"
local local2 "bar"
local abcd "foo bar"
"""
run_noecho(code, run_as_prog=run_as_program_w_locals)
run_noecho(dedent("""\
    disp `"`local1' `local2' `local3'"'
    disp `"`local1' `local2' `local3' `test1'"'
    """), run_as_prog=run_as_program_w_locals)

foo bar "3"
foo bar "3" blah blah


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()